# Dynamics of Kirchhoff Plates

$D\nabla^2\nabla^2w = q - \rho h\ddot{w}$,

with $w$ being the displacement normal to the plate, $q$ external normal stress applied along the plate, $h$ the plate thickness. Moreover

$D = \frac{Eh^3}{12(1-\nu^2)}$,

with $\rho$, $E$ and $\nu$ being the plate constitutive properties, respectively its density, Young's modulus and Poisson's ratio.

## Uniaxial deformation of rectangular plates

For rectangular plates that are deformed only along a single direction---let's say along $x$---the plates equation simplifies into the equation of motion of a Bernoulli beam:

$EI\frac{\partial^4 w}{\partial x^4} = q(r,t) - \mu \frac{\partial^2 w}{\partial t^2}$.

## Finite element approximation

The finite element method solves the plate equation of motion in its variational (or weak) form:

\begin{equation}
EI\int_0^R \Big(\partial_x^2 w(x,t) \partial_x^2\tilde{w}(x,t) dx\Big) = \int_0^R\Big(q(x,t)\tilde{w}(x,t) dx\Big) -\mu \int_0^R\Big(\ddot{w}(x,t)\tilde{w}(x,t) dx\Big),
\end{equation}

with $\tilde{w}$ being an arbitrary test function of differentiability class $C^2$ that is zero on the region of the plate with imposed-displacement boundary conditions. (I will write a separate note detailing how to obtain this variational form). 
Next the plate of interest is discretized with sampling points ($=$nodes) from which the displacement field is interpolated over space following:

$$
w(x,t) = \sum_i N_i(x)w_i(t) = \underline{N}(x)\cdot\underline{d}(t),
$$

with $\underline{d}$ a vector containing the values of the unknown field $w$ at the nodes and $\underline{N}$ the associated vector containing the interpolation functions (aka shape functions). Using the same interpolation for $\tilde{w}(x,t)= \underline{N}(x)\cdot\underline{\tilde{d}}(t)$, the variational form can be written as

\begin{equation}
EI\int_0^L \Big(\partial_x^2\underline{N}\underline{d}\partial_x^2\underline{N}\underline{\tilde{d}} dx\Big) = \int_0^L\Big(q(x,t)\underline{N}\underline{\tilde{d}} dx\Big) -\mu \int_0^L\Big(\underline{N}\underline{\ddot{d}}\underline{N}\underline{\tilde{d}} dx\Big).
\end{equation}

The displacement vectors do not depend on $r$ and could be taken out of the integral by rewritting it as:

\begin{equation}
\underline{\tilde{d}}^{\top}EI\int_0^R \Big(\partial_x^2\underline{N}^\top\partial_x^2\underline{N}dx\Big)\underline{d} = \underline{\tilde{d}}^{\top}\int_0^R\Big(\underline{N}^\top q(x,t)dx\Big) - \underline{\tilde{d}}^{\top}\mu\int_0^R\Big(\underline{N}^\top\underline{N} dx\Big)\underline{\ddot{d}}.
\end{equation}

As the test function is arbitrary, $\underline{\tilde{d}}^\top$ can be dropped in the equation above which takes then the form:
\begin{equation}
{\bf K}\underline{d} = \underline{f}_{\rm ext} - {\bf M}\underline{\ddot{d}},
\end{equation}
with ${\bf K}$ and ${\bf M}$ being respectively referred to as the stiffness and mass matrices. $\underline{f}_{\rm ext}$ is the vector of externally applied forces.

The strategy behind finite element modelling is to pick up interpolation functions that are piecewise polynomials taking finite values over some portion of the domain (aka the element) and being zero elsewhere. In the 1D geometry of interest, an element simply corresponds to a segment with two nodes at its edges. To guarantee the convergence of the numerical scheme, the choice of these polynomials must respect mainly two criteria:
1) The interpolation should be of class $C^{m-1}$ across the element boundaries.

2) The interpolation should be of class $C^{m}$ in the element and the polynomials should be complete up to degree $m$.

$m$ corresponds to the order of the variational form of the equation, $m=2$ for Kirchhoff plate equation. 
1) requires both $w$ and $\partial w/\partial r\equiv\theta$ to be continuous between two elements, which could be achieved by settings both $w$ and its first derivative $\theta$ as nodal unknowns (aka degrees of freedom). The displacements vector for an element reads then
\begin{equation}
\underline{d}^e \equiv \lbrace w_1, \theta_1, w_2, \theta_2\rbrace,
\end{equation}
with the subscript $1$ and $2$ referring respectively to the left and right nodes of the element.

2) can be satisfied by using Hermite shape functions. They correspond to third-order polynomials defined as the displacement field obtained through an element by setting the degree of freedom of interest to one and the three others to zero. (This might become clearer after looking at the next cell).

In [ ]:
import scipy.special as sp
import numpy as np
import matplotlib.pyplot as plt
import sympy as syp

In [ ]:
#Computing Hermite shape functions for one element
from sympy.abc import a,b,c,d
dofs_per_node = 2
nodes_per_elem = 2
x, xl, xr = syp.symbols('x x_l x_r')
uns = np.array([[1,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,1]])
N = []
for u in range(0,dofs_per_node*nodes_per_elem):
    shape = syp.solve_poly_system([a*xl*xl*xl+b*xl*xl+c*xl+d-uns[0,u],a*xr*xr*xr+b*xr*xr+c*xr+d-uns[1,u],3*a*xl*xl+2*b*xl+c-uns[2,u],3*a*xr*xr+2*b*xr+c-uns[3,u]], a, b, c, d)
    func = shape[0][0]*x*x*x + shape[0][1]*x*x + shape[0][2]*x + shape[0][3]
    N.append(syp.factor(func.simplify()))

In [ ]:
#Display the shape function of interest (for an element between r=1 and r=2)
shape_no = 1
syp.plot(N[shape_no].subs([(xl,1),(xr,2)]),(x,1,2),title='$N= $'+str(N[shape_no]),ylabel=None)

Once the shape functions have been properly defined, one can compute the element stiffess matrix (for the nodal positions $x=x_l$ and $x=x_r$):
\begin{equation}
{\bf K}^e \equiv EI\int_{x_l}^{x_r} \Big(\partial_x^2\underline{N}^\top\partial_x^2\underline{N}dx\Big)
\end{equation}
and the element mass matrix
\begin{equation}
{\bf M}^e \equiv \mu\int_{x_r}^{x_l} \Big(\underline{N}^\top\underline{N}dx\Big).
\end{equation}

In [ ]:
#Taking EI=1, \mu h=1 (could be readily set to dimensional values later)
k = []
m = []
for i in range(0,dofs_per_node*nodes_per_elem):
    k.append([])
    m.append([])
    for j in range(i,dofs_per_node*nodes_per_elem):
        nabla_Nt = syp.simplify(syp.diff((syp.diff(N[i],x)),x))
        nabla_N = syp.simplify(syp.diff((syp.diff(N[j],x)),x))
        k[i].append(syp.factor(syp.integrate(nabla_Nt*nabla_N,(x,xl,xr))))
        m[i].append(syp.factor(syp.integrate(N[i]*N[j],(x,xl,xr))))
Kalg = syp.Matrix(4,4,lambda i,j: k[i][j-i] if(j>=i) else k[j][i-j])
Malg = syp.Matrix(4,4,lambda i,j: m[i][j-i] if(j>=i) else m[j][i-j])

Solving the finite element problem consists then by discretizing the plate of interest with elements

In [ ]:
plate_length = 1.0
nb_nodes = 128
nodes = np.linspace(0.,plate_length,nb_nodes)
nb_elements = nb_nodes-1 

and computing the global matrices ${\bf K}$ and ${\bf M}$ as an assembly of the element matrices ${\bf K}_e$ and ${\bf M}_e$ evaluated for each element. The resulting ${\bf K}$ and ${\bf M}$ are sparse matrices of size _nb_dofs_ x _nb_dofs_.

In [ ]:
#Can take some time for fine mesh
import scipy.sparse as spa
from tqdm import tqdm
K_ref = spa.lil_matrix((nb_nodes*dofs_per_node,nb_nodes*dofs_per_node),dtype=np.float64)
M_ref = spa.lil_matrix((nb_nodes*dofs_per_node,nb_nodes*dofs_per_node),dtype=np.float64)
for e in tqdm(range(0,nb_elements)):
    n1 = e
    n2 = e+1
    for l in range(0, dofs_per_node*nodes_per_elem):
        for c in range(0, dofs_per_node*nodes_per_elem):
            K_ref[n1*dofs_per_node+l,n1*dofs_per_node+c] += syp.limit(syp.limit(Kalg[l,c], xl, nodes[n1]), xr, nodes[n2])
            M_ref[n1*dofs_per_node+l,n1*dofs_per_node+c] += syp.limit(syp.limit(Malg[l,c], xl, nodes[n1]), xr, nodes[n2])

Using this finite element approach, we now have some freedom to model different types of problems and boundary conditions.

### Example 1: modal analysis

For instance, we can repeat numerically the modal analysis presented above. Neglecting again the effect of externally applied stress and using the same decomposition $w(r,t)=W(r)\Big(C_1e^{i\omega_nt}+C_2e^{-i\omega_nt}\Big)$, the finite element equation of motion becomes:
\begin{equation}
{\bf K}\underline{d} = \omega^2{\bf M}\underline{d},
\end{equation}
which leads to the following eigenvalue problem
\begin{equation}
{\bf M}^{-1}{\bf K}\underline{d} = \omega^2\underline{d}.
\end{equation}

First, we need to apply the boundary conditions, which implies to remove the blocked degrees of freedom and their associated rows and columns in ${\bf K}$ and ${\bf M}$.

In [ ]:
def removeRowsAndColumns(smat,indexes):
    for index in np.sort(indexes)[::-1]:
        smat.rows = np.delete(smat.rows, index)
        smat.data = np.delete(smat.data, index)
        l=0
        for list in smat.rows:
            i=0
            while i<len(list):
                if(list[i]>index):
                    list[i] -= 1
                elif(list[i]==index):
                    del list[i]
                    del smat.data[l][i]
                    i-=1
                i+=1
            l+=1
        smat._shape = (smat._shape[0]-1,smat._shape[1]-1)
    return

Let's assume a cantilever situation where the beam is clamped at its left end, such that both $w$ and $\partial w/\partial x$ are zero there.

In [ ]:
K = spa.lil_matrix(K_ref,copy=True)
M = spa.lil_matrix(M_ref,copy=True)
blocked_dofs = np.array([0,1])
removeRowsAndColumns(K,blocked_dofs)
removeRowsAndColumns(M,blocked_dofs)

Then we can solve the eigenvalue problem:

In [ ]:
import scipy.sparse.linalg
#More efficient format for matrix operations
K = K.tocsc()
M = M.tocsc()
invMK = spa.linalg.inv(M)@K
#Solve for the first three modes
puls, displ = spa.linalg.eigs(invMK,k=3,sigma=0,which='LM')

and display the result:

In [ ]:
displ.shape

In [ ]:
n=1 #Mode number
index = np.argsort(puls)[n-1]
#Extract the displacements from the dofs (ignore the derivatives)
w_fem = displ[0::2,index].real/displ[-2,index].real
w_fem = np.insert(w_fem,0,0.)
fig3,ax3 = plt.subplots()
ax3.plot(nodes,w_fem,'.-g')
ax3.set_xlabel('$x/L$')
ax3.set_ylabel('$W_'+str(n)+'(x)$')
ax3.set_title('$\\omega_'+str(n)+'$=$'+str(np.sqrt(puls[index].real))+'$')

### Example 2: Dynamic simulation

Dynamic simulation implies to compute the time evolution of $\underline{d}$. First, time is sampled with a constant time step $\Delta t$, such that the equation of motion of the finite element system at the $m^{\rm th}$ time step writes:
\begin{equation}
{\bf M}\ddot{\underline{d}}^m+{\bf K}\underline{d}^m = \underline{f}^m_{\rm ext}.
\end{equation}
Time integration between $t$ and $t+\Delta t$ is then required to compute the degrees of freedom at the next time step $\underline{d}^{m+1}$. This is typically done using a Newmark time stepping scheme. See the Appendix of the supporting note for detailed information about Newmark methods.

Newmark-$\beta$ algorithm is adopted hereafter for the time integration and consists of the following steps:

i) Compute residual $\underline{r}^{m+1} = f_{\rm ext}^{m+1} - {\bf M}\ddot{\underline{d}}^{m} - {\bf K}\Big(\underline{d}^{m}+\dot{\underline{d}}^{m}\Delta t+\ddot{\underline{d}}^{m}\frac{(\Delta t)^2}{2}\Big)$

ii) Compute increment $\ddot{\delta}=\Big({\bf M}+\beta{\bf K}\frac{(\Delta t)^2}{2}\Big)^{-1}\underline{r}^{m+1}\equiv{\bf A}\underline{r}^{m+1}$

iii) Update degrees of freedom: 

$\ddot{\underline{d}}^{m+1} = \ddot{\underline{d}}^{m}+\ddot{\delta}$

$\dot{\underline{d}}^{m+1} = \dot{\underline{d}}^{m}+\Big(\ddot{\underline{d}}^{m}+\frac{1}{2}\ddot{\delta}\Big)\Delta t$

$\underline{d}^{m+1} = \underline{d}^{m}+\dot{\underline{d}}^{m}\Delta t+\Big(\ddot{\underline{d}}^{m}+\beta\ddot{\delta}\Big)\frac{(\Delta t)^2}{2}$

Note that for the linear problems of interest ${\bf M}$, ${\bf K}$ and ${\bf A}$ do not change with deformation and time integration is performed in a single execution of i), ii) and iii). For non-linear systems, those steps might have to be repeated iteratively. 

Two values of $\beta$ are usually used in pratice. $\beta=0.5$ has the advantage of being unconditionnaly stable. $\beta=0$ can be used in association to a "lumped mass matrix" (diagonal approximation of ${\bf M}$) to speed-up the Newmark scheme at the cost of a conditional stability (only small $\Delta t$ are stable).  

#### Boundary conditions

Two types of boundary conditions can be applied to every degrees of freedom of the system. Either Dirichelet boudary condition (imposed-displacement) that sets the value of $d_d$ or Neumann boundary condition (imposed-force) that sets the value of $f_n$. (Note that no boundary condition usually corresponds to a free boundary condition, i.e. $f_n=0$.)
Consequently, the unknowns to compute at each time step correspond to $d_n$ and $f_d$ Partitionning the equation of motion accordingly, one can write:
\begin{equation}
\left[
\begin{array}{c|c}
{\bf M}_{dd} & {\bf M}_{dn}\\
\hline
{\bf M}_{nd} & {\bf M}_{nn}
\end{array}\right]
\left\lbrace
\begin{array}{c}
\ddot{\underline{d}}_d\\
\hline
\ddot{\underline{d}}_n
\end{array}\right\rbrace +
\left[
\begin{array}{c|c}
{\bf K}_{dd} & {\bf K}_{dn}\\
\hline
{\bf K}_{nd} & {\bf K}_{nn}
\end{array}\right]
\left\lbrace
\begin{array}{c}
\underline{d}_d\\
\hline
\underline{d}_n
\end{array}\right\rbrace =
\left\lbrace
\begin{array}{c}
\underline{f}_d\\
\hline
\underline{f}_n
\end{array}\right\rbrace.
\end{equation}
From the linear system of equations above, one can compute the displacement unkowns by rewriting the second row as:
\begin{equation}
{\bf M}_{nn}\ddot{\underline{d}}_n + {\bf K}_{nn}\underline{d}_n = \underline{f}_n - {\bf M}_{nd}\ddot{\underline{d}}_d - {\bf K}_{nd}\underline{d}_d.
\end{equation}
Applying the boundary conditions consists then in including the effect of blocked displacements (Dirichelet B.C.) in the array of external force: $\underline{f}_{\rm ext}\equiv \underline{f}_n - {\bf M}_{nd}\ddot{\underline{d}}_d - {\bf K}_{nd}\underline{d}_d $ and removing the line and column of ${\bf M}$ and ${\bf K}$ associated to Neumann B.C before computing the Newmark integration scheme.

We could also compute the reaction forces $f_d$ associated to the imposed displacements by using the first row of the system of equations above:
\begin{equation}
\underline{f}_d =  {\bf M}_{dd}\ddot{\underline{d}}_d + {\bf M}_{dn}\ddot{\underline{d}}_n + {\bf K}_{dd}\ddot{\underline{d}}_d + {\bf K}_{dn}\ddot{\underline{d}}_n 
\end{equation}

In [ ]:
def computeExternalForce(stiff_mat,mass_mat,f_ext,dofs_to_block,imp_displ,imp_acc):
    nb_blocked = len(dofs_to_block)
    for i in range(0,nb_blocked):
        for j in stiff_mat.rows[dofs_to_block[i]]:
            if(mass_mat is not None):
                f_ext[j] -= stiff_mat[j,dofs_to_block[i]]*imp_displ[i] + mass_mat[j,dofs_to_block[i]]*imp_acc[i]       
            else:
                f_ext[j] -= stiff_mat[j,dofs_to_block[i]]*imp_displ[i]
    return

In [ ]:
def computeExternalForceNew(stiff_mat,mass_mat,f_ext,dirichelet,displ,acc):
    f_ext[dirichelet==False] -= stiff_mat[dirichelet==False,:][:,dirichelet==True]*displ[dirichelet==True] 
    f_ext[dirichelet==False] -= mass_mat[dirichelet==False,:][:,dirichelet==True]*acc[dirichelet==True]
    return

In [ ]:
#Example of plates displaced at its center, clamped at its edge and supported by a ring piston 
pis_start=0.375
pis_end=0.425

dirichelet_nodes = np.zeros(nb_nodes,dtype=bool)
dirichelet_nodes[nodes>pis_start] = True
dirichelet_nodes[nodes>pis_end] *= False

dirichelet_dofs = np.zeros(nb_nodes*dofs_per_node,dtype=bool)
dirichelet_dofs[::2] = dirichelet_nodes
#dirichelet_dofs[1::2] = dirichelet_nodes

In [ ]:
delta_t = 1e-3
nb_t_steps = 10000
beta = 0.5 #unconditionally stable

In [ ]:
K = spa.lil_matrix(K_ref,copy=True)
K_full = spa.lil_matrix(K_ref,copy=True)
M = spa.lil_matrix(M_ref,copy=True)
M_full = spa.lil_matrix(M_ref,copy=True)
fext = np.zeros(nb_nodes*dofs_per_node)
d = np.zeros(nb_nodes*dofs_per_node)
v = np.zeros(nb_nodes*dofs_per_node)
a = np.zeros(nb_nodes*dofs_per_node)
w = np.zeros((nb_nodes,nb_t_steps))
force = np.zeros((nb_nodes*dofs_per_node,nb_t_steps))
computeExternalForceNew(K,M,fext,dirichelet_dofs,d,a) #Imposed accelerations are zero
#removeRowsAndColumns(K,blocked_dofs)
#removeRowsAndColumns(M,blocked_dofs)
K = K_full[dirichelet_dofs==False,:][:,dirichelet_dofs==False]
M = M_full[dirichelet_dofs==False,:][:,dirichelet_dofs==False]
K = K.tocsc()
M = M.tocsc()
A = spa.linalg.inv(M+0.5*beta*delta_t*delta_t*K)

In [ ]:
for t in tqdm(range (0,nb_t_steps)):
    #Update of B.C.
    fext = np.zeros(nb_nodes*dofs_per_node)
    if(t<nb_t_steps//10):
        d[dirichelet_dofs==True]+= 0.2*delta_t
    computeExternalForceNew(K_full,M_full,fext,dirichelet_dofs,d,a) #Imposed accelerations are zero
        
    #Newmark-beta algorithm
    d[dirichelet_dofs==False] += v[dirichelet_dofs==False]*delta_t+0.5*a[dirichelet_dofs==False]*delta_t*delta_t
    v[dirichelet_dofs==False] += a[dirichelet_dofs==False]*delta_t   
    res = fext[dirichelet_dofs==False]-M*a[dirichelet_dofs==False]-K*d[dirichelet_dofs==False]
    incr = A*res
    a[dirichelet_dofs==False] += incr
    v[dirichelet_dofs==False] += 0.5*incr*delta_t
    d[dirichelet_dofs==False] += 0.5*beta*delta_t*delta_t*incr

    force[:,t] = K_full*d + M_full*a

    #Output
    #w[0,t] = d_0[0]
    w[:,t] = d[0::2]

In [ ]:
from matplotlib.animation import FuncAnimation
%matplotlib widget
def timeanimation(t_step):
    line.set_data(nodes, w[:,t_step])
    return line
fig,ax = plt.subplots(figsize=(10,4))
line, = ax.plot([],color="firebrick")
ax.set_xlim(0,1)
ax.set_ylim(0.,0.2)
ax.set_xlabel('Plate radius $r/R$')
ax.set_ylabel('Deflection $w/\\Omega$')
anim = FuncAnimation(fig, timeanimation, frames=nb_t_steps, interval=250)
plt.show()

In [ ]:
plt.close()

In [ ]:
d[dirichelet_dofs==True]=4.0

In [ ]:
d[dirichelet_dofs==True]

In [ ]:
a = np.linspace(0,1,10)

In [ ]:
a[a<0.5]

In [ ]:
plt.figure()
plt.plot(force[100,:])
plt.plot(force[102,:])
plt.plot(force[104,:])
plt.plot(force[106,:])
plt.plot(force[106,:])
#plt.plot(force[100,:]+force[102,:]+force[104,:]+force[106,:]+force[108,:])

In [ ]:
fft = np.fft.fft(force[106,:])
fftfrequ = np.fft.fftfreq(len(force[106,:]),1)

In [ ]:
plt.figure()
plt.plot(fftfrequ,np.abs(fft))